# <div align="center">YOLO: Real-time Object Detection</div>
---------------------------------------------------------------------

you can Find me on Github:
> ###### [ GitHub](https://github.com/lev1khachatryan)

You only look once (YOLO) is an object detection system targeted for real-time processing. We will introduce YOLO, YOLOv2 and YOLO9000 in this notebook.

Let’s start with our own testing image below.

<img src='asset/7_8/1.jpg'>

The objects detected by YOLO:

<img src='asset/7_8/2.png'>

## <div align="center">Grid cell</div>
---------------------------------------------------------------------

For our discussion, we crop our original photo. YOLO divides the input image into an S×S grid. Each grid cell predicts only one object. For example, the yellow grid cell below tries to predict the “person” object whose center (the blue dot) falls inside the grid cell.

<img src='asset/7_8/3.jpg'>

Each grid cell predicts a fixed number of boundary boxes. In this example, the yellow grid cell makes two boundary box predictions (blue boxes) to locate where the person is.

<img src='asset/7_8/4.jpg'>

However, the one-object rule limits how close detected objects can be. For that, YOLO does have some limitations on how close objects can be. For the picture below, there are 9 Santas in the lower left corner but YOLO can detect 5 only.

<img src='asset/7_8/5.jpg'>

For each grid cell

* it predicts B boundary boxes and each box has one box ***confidence score***,


* it detects one object only regardless of the number of boxes B,


* it predicts C conditional class probabilities (one per class for the likeliness of the object class).

To evaluate PASCAL VOC, YOLO uses 7×7 grids (S×S), 2 boundary boxes (B) and 20 classes (C).

<img src='asset/7_8/6.jpg'>

Let’s get into more details. Each boundary box contains 5 elements: (x, y, w, h) and a box confidence score. The confidence score reflects how likely the box contains an object (objectness) and how accurate is the boundary box. We normalize the bounding box width w and height h by the image width and height. x and y are offsets to the corresponding cell. Hence, x, y, w and h are all between 0 and 1. Each cell has 20 conditional class probabilities. The conditional class probability is the probability that the detected object belongs to a particular class (one probability per category for each cell). So, YOLO’s prediction has a shape of (S, S, B×5 + C) = (7, 7, 2×5 + 20) = (7, 7, 30).

<img src='asset/7_8/7.jpg'>

The major concept of YOLO is to build a CNN network to predict a (7, 7, 30) tensor. It uses a CNN network to reduce the spatial dimension to 7×7 with 1024 output channels at each location. YOLO performs a linear regression using two fully connected layers to make 7×7×2 boundary box predictions (the middle picture below). To make a final prediction, we keep those with high box confidence scores (greater than 0.25) as our final predictions (the right picture).

<img src='asset/7_8/8.png'>

The class confidence score for each prediction box is computed as:

<img src='asset/7_8/50.png'>

It measures the confidence on both the classification and the localization (where an object is located).

We may mix up those scoring and probability terms easily. Here are the mathematical definitions for your future reference.

<img src='asset/7_8/51.png'>

## <div align="center">Network design</div>
---------------------------------------------------------------------

<img src='asset/7_8/9.png'>

YOLO has 24 convolutional layers followed by 2 fully connected layers (FC). Some convolution layers use 1 × 1 reduction layers alternatively to reduce the depth of the features maps. For the last convolution layer, it outputs a tensor with shape (7, 7, 1024). The tensor is then flattened. Using 2 fully connected layers as a form of linear regression, it outputs 7×7×30 parameters and then reshapes to (7, 7, 30), i.e. 2 boundary box predictions per location.

A faster but less accurate version of YOLO, called Fast YOLO, uses only 9 convolutional layers with shallower feature maps.

## <div align="center">Loss function</div>
---------------------------------------------------------------------

YOLO predicts multiple bounding boxes per grid cell. To compute the loss for the true positive, we only want one of them to be responsible for the object. For this purpose, we select the one with the highest IoU (intersection over union) with the ground truth. This strategy leads to specialization among the bounding box predictions. Each prediction gets better at predicting certain sizes and aspect ratios.

YOLO uses sum-squared error between the predictions and the ground truth to calculate loss. The loss function composes of:

* the classification loss.


* the localization loss (errors between the predicted boundary box and the ground truth).


* the confidence loss (the objectness of the box).

### <div align="center">Classification Loss</div>
---------------------------------------------------------------------

If an object is detected, the classification loss at each cell is the squared error of the class conditional probabilities for each class:

<img src='asset/7_8/10.png'>

### <div align="center">Localization loss</div>
---------------------------------------------------------------------

The localization loss measures the errors in the predicted boundary box locations and sizes. We only count the box responsible for detecting the object.

<img src='asset/7_8/11.png'>

We do not want to weight absolute errors in large boxes and small boxes equally. i.e. a 2-pixel error in a large box is the same for a small box. To partially address this, YOLO predicts the square root of the bounding box width and height instead of the width and height. In addition, to put more emphasis on the boundary box accuracy, we multiply the loss by λcoord (default: 5).

### <div align="center">Confidence loss</div>
---------------------------------------------------------------------

If an object is detected in the box, the confidence loss (measuring the objectness of the box) is:

<img src='asset/7_8/12.png'>

If an object is not detected in the box, the confidence loss is:

<img src='asset/7_8/13.png'>

Most boxes do not contain any objects. This causes a class imbalance problem, i.e. we train the model to detect background more frequently than detecting objects. To remedy this, we weight this loss down by a factor $λ_{noobj}$ (default: 0.5).

## <div align="center">Loss</div>
---------------------------------------------------------------------

The final loss adds localization, confidence and classification losses together.

<img src='asset/7_8/14.png'>

# <div align="center">Inference: Non-maximal suppression</div>
---------------------------------------------------------------------

YOLO can make duplicate detections for the same object. To fix this, YOLO applies non-maximal suppression to remove duplications with lower confidence. Non-maximal suppression adds 2- 3% in mAP.

Here is one of the possible non-maximal suppression implementation:

* Sort the predictions by the confidence scores.


* Start from the top scores, ignore any current prediction if we find any previous predictions that have the same class and IoU > 0.5 with the current prediction.


* Repeat step 2 until all predictions are checked.

# <div align="center">Benefits of YOLO</div>
---------------------------------------------------------------------

* Fast. Good for real-time processing.


* Predictions (object locations and classes) are made from one single network. Can be trained end-to-end to improve accuracy.


* YOLO is more generalized. It outperforms other methods when generalizing from natural images to other domains like artwork.

<img src='asset/7_8/15.png'>

* Region proposal methods limit the classifier to the specific region. YOLO accesses to the whole image in predicting boundaries. With the additional context, YOLO demonstrates fewer false positives in background areas.


* YOLO detects one object per grid cell. It enforces spatial diversity in making predictions.

# <div align="center">YOLOv2</div>
---------------------------------------------------------------------

SSD is a strong competitor for YOLO which at one point demonstrates higher accuracy for real-time processing. Comparing with region based detectors, YOLO has higher localization errors and the recall (measure how good to locate all objects) is lower. YOLOv2 is the second version of the YOLO with the objective of improving the accuracy significantly while making it faster.

## <div align="center">Accuracy improvements</div>
---------------------------------------------------------------------

### Batch normalization

Add batch normalization in convolution layers. This removes the need for dropouts and pushes mAP up 2%.

### High-resolution classifier

The YOLO training composes of 2 phases. First, we train a classifier network like VGG16. Then we replace the fully connected layers with a convolution layer and retrain it end-to-end for the object detection. YOLO trains the classifier with 224 × 224 pictures followed by 448 × 448 pictures for the object detection. YOLOv2 starts with 224 × 224 pictures for the classifier training but then retune the classifier again with 448 × 448 pictures using much fewer epochs. This makes the detector training easier and moves mAP up by 4%.

### Convolutional with Anchor Boxes

As indicated in the YOLO paper, the early training is susceptible to unstable gradients. Initially, YOLO makes arbitrary guesses on the boundary boxes. These guesses may work well for some objects but badly for others resulting in steep gradient changes. In early training, predictions are fighting with each other on what shapes to specialize on.

<img src='asset/7_8/16.jpg'>

In the real-life domain, the boundary boxes are not arbitrary. Cars have very similar shapes and pedestrians have an approximate aspect ratio of 0.41.

<img src='asset/7_8/17.jpg'>

Since we only need one guess to be right, the initial training will be more stable if we start with diverse guesses that are common for real-life objects.

<img src='asset/7_8/18.jpg'>

For example, we can create 5 anchor boxes with the following shapes.

<img src='asset/7_8/19.jpg'>

Instead of predicting 5 arbitrary boundary boxes, we predict offsets to each of the anchor boxes above. If we constrain the offset values, we can maintain the diversity of the predictions and have each prediction focuses on a specific shape. So the initial training will be more stable.

# <div align="center">Here are the changes we make to the network:</div>
---------------------------------------------------------------------

* Remove the fully connected layers responsible for predicting the boundary box.

<img src='asset/7_8/20.jpg'>

* We move the class prediction from the cell level to the boundary box level. Now, each prediction includes 4 parameters for the boundary box, 1 box confidence score (objectness) and 20 class probabilities. i.e. 5 boundary boxes with 25 parameters: 125 parameters per grid cell. Same as YOLO, the objectness prediction still predicts the IOU of the ground truth and the proposed box.

<img src='asset/7_8/21.png'>

* To generate predictions with a shape of 7 × 7 × 125, we replace the last convolution layer with three 3 × 3 convolutional layers each outputting 1024 output channels. Then we apply a final 1 × 1 convolutional layer to convert the 7 × 7 × 1024 output into 7 × 7 × 125. (See the section on DarkNet for the details.)

<img src='asset/7_8/22.jpg'>

* Change the input image size from 448 × 448 to 416 × 416. This creates an odd number spatial dimension (7×7 v.s. 8×8 grid cell). The center of a picture is often occupied by a large object. With an odd number grid cell, it is more certain on where the object belongs.

<img src='asset/7_8/23.jpg'>

* Remove one pooling layer to make the spatial output of the network to 13×13 (instead of 7×7).

Anchor boxes decrease mAP slightly from 69.5 to 69.2 but the recall improves from 81% to 88%. i.e. even the accuracy is slightly decreased but it increases the chances of detecting all the ground truth objects.

# <div align="center">Dimension Clusters</div>
---------------------------------------------------------------------

In many problem domains, the boundary boxes have strong patterns. For example, in the autonomous driving, the 2 most common boundary boxes will be cars and pedestrians at different distances. To identify the top-K boundary boxes that have the best coverage for the training data, we run K-means clustering on the training data to locate the centroids of the top-K clusters.

<img src='asset/7_8/24.jpg'>
<div align="center">(Image modified form a k-means cluster)</div>

Since we are dealing with boundary boxes rather than points, we cannot use the regular spatial distance to measure datapoint distances. No surprise, we use IoU.

<img src='asset/7_8/25.png'>

On the left, we plot the average IoU between the anchors and the ground truth boxes using different numbers of clusters (anchors). As the number of anchors increases, the accuracy improvement plateaus. For the best return, YOLO settles down with 5 anchors. On the right, it displays the 5 anchors’ shapes. The purplish-blue rectangles are selected from the COCO dataset while the black border rectangles are selected from the VOC2007. In both cases, we have more thin and tall anchors indicating that real-life boundary boxes are not arbitrary.

Unless we are comparing YOLO and YOLOv2, we will reference YOLOv2 as YOLO for now.

# <div align="center">Direct location prediction</div>
---------------------------------------------------------------------

We make predictions on the offsets to the anchors. Nevertheless, if it is unconstrained, our guesses will be randomized again. YOLO predicts 5 parameters (tx, ty, tw, th, and to) and applies the sigma function to constraint its possible offset range.